In [1]:
import sys
# sys.path.append('../qick/qick_lib/')
sys.path.append('../soft/')
import Scan
from mkids import *
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
import datetime
today = datetime.datetime.today()
import csv
from itertools import zip_longest

In [2]:
board = Scan.getBoard()

if board == 'zcu111':
    bitfile = 'mkids_2x2_kidsim_v1' 
elif board == 'zcu216':
    bitfile = 'mkids_2x2_kidsim_v2'
elif board == 'rfsoc4x2':
    bitfile = 'mkids_v3'
else:
    raise Exception("Do not know what to do with %s"%board)
scan = Scan.Scan(bitfile, iKids=0, iSimu=0)

In [3]:
# Print out properties of the firmware
print("       Nyquist Frequency:  %.1f MHz"%scan.fNyquist)
print("      Number of channels:  nInCh=%d  nOutCh=%d"%(scan.nInCh, scan.nOutCh))
print("      Channel Separation:  %.1f MHz"%scan.dfChannel)
print("              decimation:  %d"%scan.kidsChain.analysis.decimation)
print("Sampling frequency fs_ch:  %.1f MHz"%scan.kidsChain.analysis.fs_ch)


       Nyquist Frequency:  2457.6 MHz
      Number of channels:  nInCh=64  nOutCh=64
      Channel Separation:  19.2 MHz
              decimation:  2
Sampling frequency fs_ch:  19.2 MHz


In [4]:
fMixer = 2505

scan.set_mixer(fMixer)

scan.simuChain.analysis.qout(2)
scan.simuChain.synthesis.qout(2)
scan.kidsChain.analysis.qout(2)
scan.kidsChain.synthesis.qout(2)

kids = scan.kidsChain
streamer = getattr(kids.soc, kids.analysis.dict['chain']['streamer'])

In [5]:
# Place one tone in each PFB channel
fTones = scan.outCh2Freq(np.arange(scan.nInCh)+0.1)

# fTones = fTones[::8] # one tone in each transfer group

decimation = 11

In [6]:
kids.set_decimation(decimation)

phases = np.zeros(len(fTones))
gains = 0.99*np.ones(len(fTones))/len(fTones)
cgs = None

kids.set_tones(fTones, phases, gains, cgs, verbose=False)

In [8]:
scan.kidsChain.ntrans

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2,
       2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5,
       5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7])

In [7]:
d = streamer.get_data_all()['samples']
for i, f in enumerate(fTones):
    nt = kids.ntrans[i]
    idx = kids.idxs[i]
    print(i, nt, idx, d[nt][2*idx:2*idx+2].shape[1])

0 0 0 1268
1 0 1 1268
2 0 2 1268
3 0 3 1268
4 0 4 1268
5 0 5 1268
6 0 6 1268
7 0 7 1268
8 1 0 1248
9 1 1 1248
10 1 2 1248
11 1 3 1248
12 1 4 1248
13 1 5 1248
14 1 6 1248
15 1 7 1248
16 2 0 1247
17 2 1 1247
18 2 2 1247
19 2 3 1247
20 2 4 1247
21 2 5 1247
22 2 6 1247
23 2 7 1247
24 3 0 1247
25 3 1 1247
26 3 2 1247
27 3 3 1247
28 3 4 1247
29 3 5 1247
30 3 6 1247
31 3 7 1247
32 4 0 1247
33 4 1 1247
34 4 2 1247
35 4 3 1247
36 4 4 1247
37 4 5 1247
38 4 6 1247
39 4 7 1247
40 5 0 1247
41 5 1 1247
42 5 2 1247
43 5 3 1247
44 5 4 1247
45 5 5 1247
46 5 6 1247
47 5 7 1247
48 6 0 1248
49 6 1 1248
50 6 2 1248
51 6 3 1248
52 6 4 1248
53 6 5 1248
54 6 6 1248
55 6 7 1248
56 7 0 1248
57 7 1 1248
58 7 2 1248
59 7 3 1248
60 7 4 1248
61 7 5 1248
62 7 6 1248
63 7 7 1248


In [9]:
xs = scan.kidsChain.get_xs(mean=False)

for i, x in enumerate(xs):
    print(i, x.shape)

0 (1250,)
1 (1250,)
2 (1250,)
3 (1250,)
4 (1250,)
5 (1250,)
6 (1250,)
7 (1250,)
8 (1250,)
9 (1250,)
10 (1250,)
11 (1250,)
12 (1250,)
13 (1250,)
14 (1250,)
15 (1250,)
16 (1250,)
17 (1250,)
18 (1250,)
19 (1250,)
20 (1250,)
21 (1250,)
22 (1250,)
23 (1250,)
24 (1250,)
25 (1250,)
26 (1250,)
27 (1250,)
28 (1250,)
29 (1250,)
30 (1250,)
31 (1250,)
32 (1250,)
33 (1250,)
34 (1250,)
35 (1250,)
36 (1250,)
37 (1250,)
38 (1250,)
39 (1250,)
40 (1250,)
41 (1250,)
42 (1250,)
43 (1250,)
44 (1250,)
45 (1250,)
46 (1250,)
47 (1250,)
48 (1250,)
49 (1250,)
50 (1250,)
51 (1250,)
52 (1250,)
53 (1250,)
54 (1250,)
55 (1250,)
56 (1250,)
57 (1250,)
58 (1250,)
59 (1250,)
60 (1250,)
61 (1250,)
62 (1250,)
63 (1250,)
